# Silhouette Score for Allocation Algorithms

> **1.** Create dataframe from MySQL table containing allocated system

> **2.** Calculate Silhouette Score for allocation clusters

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

In [2]:
def buildDataFrame(table):
    connection = mysql.connector.connect(
                host = "localhost",
                port = 3306,
                user = 'root',
                database = "expandinghorizons",
                password = "dB79@dG2024!"
    )

    cursor = connection.cursor()

    query = ("SELECT distinct child_id FROM child ")
    cursor.execute(query)

    child_ids = cursor.fetchall()

    data_list = []
    for row in child_ids:
        data = {}
        data["id"] = row[0]
        child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              f"FROM {table} "
                              f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child1_child_query)
        child1_child = cursor.fetchall()

        for row2 in child1_child:
            # Add columns with prefixes
            data[f'{row2[1]}_distance'] = row2[2]
            data[f'{row2[1]}_time'] = row2[3]

        child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                              f"FROM {table} "
                              f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_child2_query)
        child_child2 = cursor.fetchall()

        for row3 in child_child2:
            data[f'{row3[0]}_distance'] = row3[2]
            data[f'{row3[0]}_time'] = row3[3]

        child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                          f"FROM {table} "
                          f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
        cursor.execute(child_ei_query)
        child_ei = cursor.fetchall()

        for row4 in child_ei:
            data[f'{row4[1]}_distance'] = row4[2]
            data[f'{row4[1]}_time'] = row4[3]

        assigned_query = ("SELECT ei_id_1 "
                          f"FROM {table} "
                          f"WHERE child_id_1 = '{row[0]}' and assigned = 1")
        cursor.execute(assigned_query)
        assigned = cursor.fetchall()

        data["assigned_ei"] = assigned[0][0]

        data_list.append(data)

    query = ("SELECT ei_id FROM ei")
    cursor.execute(query)

    ei_ids = cursor.fetchall()
    for row5 in ei_ids:
        data["id"] = row5[0]
        ei_child_query = ("SELECT ei_id_1, child_id_1, driving_distance, driving_time "
                          f"from {table} "
                          f"WHERE ei_id_1 = '{row5[0]}' and child_id_2 is NULL and ei_id_2 is NULL")
        cursor.execute(ei_child_query)
        ei_child = cursor.fetchall()
        for row6 in ei_child:
            data[f'{row6[1]}_distance'] = row6[2]
            data[f'{row6[1]}_time'] = row6[3]

        ei1_ei_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                        f"from {table} "
                        f"WHERE ei_id_1 = '{row5[0]}' and child_id_1 is NULL and child_id_2 is NULL")
        cursor.execute(ei1_ei_query)
        ei1_ei = cursor.fetchall()
        for row7 in ei1_ei:
            data[f'{row7[1]}_distance'] = row7[2]
            data[f'{row7[1]}_time'] = row7[3]

        ei_ei2_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                        f"from {table} "
                        f"WHERE ei_id_2 = '{row5[0]}' and child_id_1 is NULL and child_id_2 is NULL")
        cursor.execute(ei_ei2_query)
        ei_ei2 = cursor.fetchall()
        for row8 in ei1_ei:
            data[f'{row8[0]}_distance'] = row8[2]
            data[f'{row8[0]}_time'] = row8[3]

        data["assigned_ei"] = row5[0]

        data_list.append(data)

    cursor.close()
    connection.close()

    df = pd.DataFrame(data_list)

    return df

In [3]:
def processData(df):
    df = df.fillna(0)
    popped_column = df.pop("assigned_ei")
    df["assigned_ei"] = popped_column
    df = df.drop(columns=["id"])
    
    Y = df["assigned_ei"].values
    X = df.drop(["assigned_ei"], axis=1)
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    
    silhouette_avg = silhouette_score(X_scaled, Y)
    
    return silhouette_avg

In [4]:
def runSilhouette(table):
    data = buildDataFrame(table)
    silhouette_score = processData(data)

    print(f'Silhouette Score: {silhouette_score}')

In [5]:
runSilhouette("locationdatarandom")

Silhouette Score: -0.16580413570227015


In [6]:
runSilhouette("locationdatareal")

IndexError: list index out of range

In [7]:
runSilhouette("locationdataalgo1")

Silhouette Score: -0.044230672213201455


In [12]:
runSilhouette("locationdataalgo2")

Silhouette Score: 0.09451106914980036


In [9]:
runSilhouette("locationdataalgo3")

Silhouette Score: 0.12161702772837982


In [10]:
runSilhouette("locationdataalgo4")

Silhouette Score: 0.16918789262218806


In [11]:
runSilhouette("locationdataalgo5")

Silhouette Score: 0.13319871104189018
